## Parquet Dataset Generation

Generate a minimized version of the `2024.01.23_Mastertable_with Codebook-Header.xlsx` as a foundation for further analysis.

Manual Changes:
- Removed `s. T13` values from `RAI_T20`
- Removed `EU1_NT_norm_T22` entry for HD66, as it was to large for a long...
- Removed `SARS-IgG_T20` and `SARS-IgG_T20_dicho` for `C364` as they made no sense.

Change Protocol:
- [X] Remove duplicate header row and second ID row
- [X] Remove unnamed tailing columns (not required for analysis after medical consultation)
- [X] Remove columns without any values 
    - ['2_Impfung', '3_Impfung', '4_Impfung', '5_Impfung', 'IS_2_Blut', 'T25_Abstand_zur_Infektion', 'SFU_norm_T34', 'SFUsub_norm_T34', 'SFU_norm_T35', 'SFUsub_norm_T35']
- [X] Remove rows without any values
    - ['C72', 'C78', 'C80', 'C83', 'C86', 'C88', 'C92', 'C97', 'C104', 'C110', 'C112', 'C113', 'C119', 'C122', 'C123', 'C125', 'C142', 'C143', 'C147', 'C148', 'C152', 'C155', 'C159', 'C164', 'C167', 'C176', 'C179', 'C180', 'C183', 'C184', 'C186', 'C193', 'C202', 'C207', 'C208', 'C209', 'C211', 'C214', 'C215', 'C217', 'C219', 'C222', 'C226', 'C230', 'C236', 'C238', 'C241', 'C243', 'C247', 'C253', 'C254', 'C255', 'C257', 'C259', 'C260', 'C261', 'C264', 'C265', 'C267', 'C268', 'C270', 'C271', 'C274', 'C277', 'C278', 'C282', 'C283', 'C287', 'C288', 'C289', 'C292', 'C293', 'C294', 'C295', 'C297', 'C298', 'C299', 'C302', nan, 'C321', 'C322', 'C323']
- [ ] Discuss missing data
- [X] Fix encodings
- [X] Parse dtypes
    - [X] Some integer values also had an accompanying date, this was removed
    - [X] Some integer values were just `?`, these were set to nan
- [X] Split into target and control data frame
- [X] Save data frames to parquet

In [ ]:
# First, set current working directory to content root!
import os
os.chdir('../..')

# check if current working directory is now project root
assert os.getcwd().split('/')[-1] == 'immun-ml'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
raw = pd.read_excel('data/2024.01.23_Mastertable_with Codebook-Header.xlsx', sheet_name='CV-D&C', index_col='ID_ML')

### Remove second header row and ID row

In [ ]:
raw = raw.drop(['ID_ML'])
raw = raw.drop(['ID'], axis=1)

### Remove unnamed tailing columns

In [ ]:
raw = raw.loc[:, ~raw.columns.str.contains('^Unnamed')]

### Remove empty columns

In [ ]:
empty_columns = []
for col in raw.columns:
    if raw[col].dropna().empty:
        empty_columns.append(col)

raw = raw.drop(empty_columns, axis=1)

In [ ]:
empty_rows = []
for id, row in raw.iterrows():
    if row.dropna().empty:
        empty_rows.append(id)

raw = raw.drop(empty_rows, axis=0)

### Talk about missing data

In [ ]:
missing_10 = []
missing_100 = []
missing_200 = []
missing_over_200 = []
for col in raw.columns:
    nans = raw[col].isna().sum()
    if nans < 10:
        missing_10.append(col)
    elif nans < 100:
        missing_100.append(col)
    elif nans < 200:
        missing_200.append(col)
    elif nans > 200:
        missing_over_200.append(col)

In [ ]:
print('Missing less than 10:', len(missing_10))
print('Missing less than 100:', len(missing_100))
print('Missing less than 200:', len(missing_200))
print('Missing over 200:', len(missing_over_200))

### Fix encodings
`Art_der_Nierenerkrankung` is only encoded for half the values, the rest has a string of the actual kidney disease. These are also not defined in the code book.
To reduce the complexity and enable a easier processing the column is set to 1 if a disease is present and nan otherwise.

`Andere_Tumorerkrg` and `Diabetes` als does not fit the code book. Here strings can simply be set to 1 while 1, 0 and nan can be kept.

`IS_Dosis` contains random ?s as values. These are set to nan as no information can be conducted form them.

`SARS-IgG_T4` and `RAI_T4` contained `kein Serum` entries. These are removed the same way as the ?s above.

C371 has set `SARS-IgG_T14` to `>5250`. It is set to 5250.

In [ ]:
def clean_kidney_disease(row):
    if pd.isna(row):
        return row
    if type(row) == float or type(row) == int or type(row) == str:
        return 1
    return np.nan

raw['Art_der_Nierenerkrankung'] = raw['Art_der_Nierenerkrankung'].apply(lambda x: clean_kidney_disease(x))

In [ ]:
def set_string_to_1(row):
    if pd.isna(row) or row == '0':
        return row
    if row == 1 or type(row) == str:
        return 1
    return np.nan

raw['Andere_Tumorerkrg'] = raw['Andere_Tumorerkrg'].apply(lambda x: set_string_to_1(x))
raw['Diabetes'] = raw['Diabetes'].apply(lambda x: set_string_to_1(x))

In [ ]:
def remove_unwanted_string_values(row):
    if type(row) == str:
        return np.nan
    return row

raw['IS_Dosis'] = raw['IS_Dosis'].apply(lambda x: remove_unwanted_string_values(x))
raw['RAI_T4'] = raw['RAI_T4'].apply(lambda x: remove_unwanted_string_values(x))
raw['SARS-IgG_T4'] = raw['SARS-IgG_T4'].apply(lambda x: remove_unwanted_string_values(x))

In [ ]:
raw.loc['C371', 'SARS-IgG_T14'] = 5250

### Parse dtypes
Currently only handles a few dtypes, this is way to tedious

In [ ]:
## SARS-Infektion has some dates instead of ints
## these have to be formatted to ints for the parquet save to work
def remove_date_from_int_row(row):
    """
    Some int rows have a date after the integer value.
    This function strips the date.
    """
    if type(row) == str:
        row = row.split(' ')[0]
    if row == '?':
        return np.nan
    return row

raw['SARS-Infektion'] = raw['SARS-Infektion'].apply(lambda x: remove_date_from_int_row(x))
raw['1_Impfstoff'] = raw['1_Impfstoff'].apply(lambda x: remove_date_from_int_row(x))
raw['2_Impfstoff'] = raw['2_Impfstoff'].apply(lambda x: remove_date_from_int_row(x))
raw['3_Impfstoff'] = raw['3_Impfstoff'].apply(lambda x: remove_date_from_int_row(x))
raw['4_Impfstoff'] = raw['4_Impfstoff'].apply(lambda x: remove_date_from_int_row(x))
raw['5_Impfstoff'] = raw['5_Impfstoff'].apply(lambda x: remove_date_from_int_row(x))
raw['SFUsub_VZV_T4'] = raw['SFUsub_VZV_T4'].apply(lambda x: x if type(x) == float else np.nan)

In [ ]:
float_cols = [
    'Alter', 'Geschlecht', 'Dialyse', 'Gruppen', 'BTI_prevacc1', 'SARS-Infektion',
    'BTI_prevacc2', 'BTI_prevacc3', 'BTI_prevacc4', 'BTI_prevacc5', 'BTI_preT1',
    'BTI_preT2', 'BTI_preT3', 'BTI_preT4', 'BTI_preT12', 'SFUsub_VZV_T4',
    '1_Impfstoff', '2_Impfstoff', '3_Impfstoff', '4_Impfstoff', '5_Impfstoff',]

for col in float_cols:
    raw[col] = raw[col].astype(float)

date_cols = ['1_Infektion', '2_Infektion', 'Entnahme_T32']
for col in date_cols:
    raw[col] = pd.to_datetime(raw[col], errors='coerce')

### Split dataset into target and control group

In [ ]:
# split into target and control df
target_ds = raw[raw['Dialyse'] == 1]
control_ds = raw[raw['Dialyse'] == 0]

In [ ]:
print('raw rows: ', len(raw))
print('target rows: ', len(target_ds))
print('control rows: ', len(control_ds))
print('combined rows: ', len(target_ds) + len(control_ds))
print('missing rows: ', len(raw) - len(target_ds) - len(control_ds))

In [ ]:
# get all rows which are in raw but not in target or control
missing = raw[~raw.index.isin(target_ds.index) & ~raw.index.isin(control_ds.index)]
len(missing)

In [ ]:
# missing.index.to_list()

In [ ]:
# save to parquet
target_ds.to_parquet('data/immun-ml_targets.parquet')
control_ds.to_parquet('data/immun-ml_control.parquet')